In [14]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [15]:
# Store the CSV you saved created in part one into a DataFrame
city_data_df=pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Date
0,0,Ponta Do Sol,32.67,-17.10,62.60,82,40,11.41,PT,light intensity shower rain,2020-12-09 16:07:19
1,1,Punta Arenas,-53.15,-70.92,55.40,54,75,20.80,CL,broken clouds,2020-12-09 16:10:45
2,2,Lebu,-37.62,-73.65,59.68,63,0,17.02,CL,clear sky,2020-12-09 16:13:38
3,3,Nizhniy Kuranakh,58.84,125.49,2.01,92,100,2.86,RU,overcast clouds,2020-12-09 16:13:39
4,4,Puerto Ayora,-0.74,-90.35,77.00,78,75,11.41,EC,broken clouds,2020-12-09 16:13:39


In [16]:
city_data_df["Max Temp"].dtypes

dtype('float64')

In [17]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [18]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig=gmaps.figure(center=(30.00, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Heatmap of percent humidity
locations=city_data_df[["Lat", "Lng"]]
humidity=city_data_df["Humidity"]
fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Heatmap of percent cloudiness
locations=city_data_df[["Lat", "Lng"]]
clouds=city_data_df["Cloudiness"]
fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Heatmap of percent wind speed
locations=city_data_df[["Lat", "Lng"]]
wind_speed=city_data_df["Wind Speed"]
fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Ask the customer to add a minimum and maximum temperature value
min_temp=float(input("What is the minimum temperature you would like for your trip?"))
max_temp=float(input("What is the maximum temperature you would like for your trip?"))

In [ ]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

In [ ]:
preferred_cities_df.count()

In [ ]:
# Create DataFrame called hotel_df to store hotel names along with city, countries, max temp, and coordinates.
hotel_df=preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"]=""
hotel_df.head(10)


In [ ]:
# Set parameters to search for a hotel.
params= {
    "radius": 5000,
    "type": "lodging",
    "key":g_key
}

In [ ]:

# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
# Get the latitude and longitude
    lat=row["Lat"]
    lng=row["Lng"]

# Add the latitude and longitude to location key for the params dictionary
params["location"]=f"{lat},{lng}"
# Use the search term: "lodging" and our latitude and longitude
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#Make requests and get the JSON data from the search.
hotels= requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name
try:
    hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found...skipping.")

In [ ]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

In [ ]:
info_box_template="""
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig